In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import regularizers

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [3]:
train_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
train_df.keys()

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [4]:
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


# **Data Processing**

In [5]:
train_df = train_df.drop(columns=['PassengerId', 'Name'])
test_df = test_df.drop(columns=['PassengerId', 'Name'])

In [6]:
# Cabin Column
train_df[["Cabin1","Cabin2","Cabin3"]]= train_df["Cabin"].str.split("/", expand = True)
train_df["Cabin2"] = train_df["Cabin2"].astype(float)

test_df[["Cabin1","Cabin2","Cabin3"]]= test_df["Cabin"].str.split("/", expand = True)
test_df["Cabin2"] = test_df["Cabin2"].astype(float)

train_df = train_df.drop(columns=['Cabin'])
test_df = test_df.drop(columns=['Cabin'])

In [7]:
train_df.isna().sum()

HomePlanet      201
CryoSleep       217
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
Cabin1          199
Cabin2          199
Cabin3          199
dtype: int64

In [8]:
object_cols = [col for col in train_df.keys() if train_df[col].dtypes == 'object']
float_cols = [col for col in train_df.keys() if train_df[col].dtypes == 'float64']
train_df.dtypes

HomePlanet       object
CryoSleep        object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Transported        bool
Cabin1           object
Cabin2          float64
Cabin3           object
dtype: object

In [9]:
# Replace NA Values in Object Datatype Columns with random values
for col in object_cols:
    unique_vals = train_df[col].unique().tolist()
    unique_vals = [item for item in unique_vals if str(item) != 'nan']
    train_df[col] = train_df[col].fillna(unique_vals[np.random.randint(0,len(unique_vals))])
    
# Replace NA Values in Float64 Datatype Columns with random values
for col in float_cols:
    avg_col = train_df[col].mean()
    train_df[col] = train_df[col].fillna(avg_col)   

In [10]:
# Convert Object dtype columns
for col in object_cols:
    test_df[col].replace(train_df[col].unique(), range(len(train_df[col].unique())), inplace=True)
    train_df[col].replace(train_df[col].unique(), range(len(train_df[col].unique())), inplace=True)
    
train_df['Transported'].replace(train_df['Transported'].unique(), range(len(train_df['Transported'].unique())), inplace=True)

In [11]:
# Switch column order to put'Transported' to the end
train_df = train_df[list(test_df.columns.values) + ['Transported']]

In [12]:
train_df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1,Cabin2,Cabin3,Transported
0,0,0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
1,1,0,0,24.0,0,109.0,9.0,25.0,549.0,44.0,1,0.0,1,1
2,0,0,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,2,0.0,1,0
3,0,0,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,2,0.0,1,0
4,1,0,0,16.0,0,303.0,70.0,151.0,565.0,2.0,1,1.0,1,1


In [13]:
test_df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1,Cabin2,Cabin3
0,1.0,1.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0
1,1.0,0.0,0.0,19.0,0.0,0.0,9.0,0.0,2823.0,0.0,1.0,4.0,1.0
2,0.0,1.0,2.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0
3,0.0,0.0,0.0,38.0,0.0,0.0,6652.0,0.0,181.0,585.0,6.0,1.0,1.0
4,1.0,0.0,0.0,20.0,0.0,10.0,0.0,635.0,0.0,0.0,1.0,5.0,1.0


In [14]:
train_df = train_df.sample(frac = 1)
SPLIT_SIZE = 0.7

data = train_df.iloc[:,:-1].to_numpy()
labels = train_df.pop('Transported').to_numpy()

train_size = int(data.shape[0]*SPLIT_SIZE)
train_data = data[:train_size]
train_labels = labels[:train_size]

val_data = data[train_size:]
val_labels = labels[train_size:]

# **Model**

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

2022-11-26 12:00:06.656835: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [16]:
model.fit(train_data, train_labels, epochs=40, batch_size=64, validation_data=(val_data, val_labels))

2022-11-26 12:00:06.998487: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/40
96/96 [==============================] - 2s 10ms/step - loss: 4.2383 - accuracy: 0.7006 - val_loss: 3.3545 - val_accuracy: 0.6039
Epoch 2/40
96/96 [==============================] - 1s 6ms/step - loss: 2.0416 - accuracy: 0.7185 - val_loss: 4.8889 - val_accuracy: 0.5936
Epoch 3/40
96/96 [==============================] - 1s 6ms/step - loss: 2.0761 - accuracy: 0.7272 - val_loss: 1.4479 - val_accuracy: 0.7327
Epoch 4/40
96/96 [==============================] - 1s 6ms/step - loss: 1.2422 - accuracy: 0.7349 - val_loss: 0.9946 - val_accuracy: 0.7400
Epoch 5/40
96/96 [==============================] - 1s 5ms/step - loss: 1.0972 - accuracy: 0.7267 - val_loss: 0.9630 - val_accuracy: 0.7262
Epoch 6/40
96/96 [==============================] - 1s 6ms/step - loss: 0.9476 - accuracy: 0.7408 - val_loss: 0.9378 - val_accuracy: 0.7715
Epoch 7/40
96/96 [==============================] - 1s 6ms/step - loss: 0.8940 - accuracy: 0.7351 - val_loss: 0.8158 - val_accuracy: 0.7393
Epoch 8/40
96/96 [=

# **Predictions**

In [17]:
x = test_df.to_numpy()
#x = np.asarray(x).astype('float64')
predictions = model.predict(x)

In [18]:
predictions = [1 if pred >= 0.5 else 0 for pred in predictions]

In [19]:
id_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

result = pd.DataFrame(columns=['PassengerId', 'Transported'])
result['PassengerId'] = [id for id in id_df['PassengerId']]
result['Transported'] = predictions
result['Transported'].replace([0, 1], ['False', 'True'], inplace=True)

In [20]:
result.to_csv('submission1.csv', index=False)